In [2]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
import re

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

In [12]:
soup = bs(browser.html, 'lxml')

article = soup.find('div', class_ = 'list_text')

title = article.find('a').text

summary = article.find('div', class_ = "article_teaser_body").text

print(title)
print(summary)

NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound
The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet.


# Featured image stuff

In [13]:
featuredImageUrl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featuredImageUrl)

In [18]:
imageSoup = bs(browser.html, 'lxml')

featuredImgUrl = 'https://www.jpl.nasa.gov/' + imageSoup.find('footer').find('a')['data-fancybox-href']
print(featuredImgUrl)

https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA09178_ip.jpg


# Twitter stuff

In [4]:
twitterUrl = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitterUrl)

In [10]:
twitterSoup = bs(browser.html, 'lxml')

tweets = twitterSoup.find_all('div', attrs = {'data-testid': 'tweet'})
for tweet in tweets:
    weather = tweet.find('div', class_ = 'r-bnwqim')
    weatherText = weather.find('span').text
    if 'InSight' in weatherText:
        break

print (weatherText)

InSight sol 532 (2020-05-26) low -93.1ºC (-135.7ºF) high -1.1ºC (30.0ºF)
winds from the SW at 4.9 m/s (10.9 mph) gusting to 17.7 m/s (39.6 mph)
pressure at 7.10 hPa


# Mars facts

In [56]:
factsUrl = 'https://space-facts.com/mars/'

facts = pd.read_html(factsUrl)
facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
                            0                                 1
 0                       Age:                 4.6 Billion Years
 1                      Type:                Yellow Dwarf (G2V)
 2                  Diameter:                      1,392,684 km
 3  Equatorial Circumference:                    4,370,005.6 km
 4                      Mass:  1.99 × 10^30 kg (333,060 Earths)
 5       Surface Temperature:                          

In [57]:
dfFacts = facts[0]

dfFacts.columns = ['Stat', 'Value']
dfFacts.set_index('Stat', inplace = True)

dfFacts.head()

,Value
Stat,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [58]:
factsHtml = dfFacts.to_html()
factsHtml

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Stat</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tb

# Mars Hemispheres

In [67]:
hemiUrl = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemiUrl)

hemiSoup = bs(browser.html, 'lxml')

In [68]:
lHemispheres = []

hemispheres = hemiSoup.find_all('div', class_ = 'description')

baseUrl = 'https://astrogeology.usgs.gov'

for hemi in hemispheres:
    browser.visit(hemiUrl)
    
    dictHemi = {}
    dictHemi['title'] = (hemi.find('h3').text).replace(' Enhanced', '')
    
    browser.click_link_by_partial_text(dictHemi['title'])
    
    hemiSoup2 = bs(browser.html, 'lxml')
    
    dictHemi['img_url'] = hemiSoup2.find('div', class_ = 'downloads').find('a')['href']
    
    lHemispheres.append(dictHemi)
    
print(lHemispheres)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
